# Ollama Tutorial - Local LLMs

This notebook covers running LLMs locally on your machine using Ollama and the `llm_playbook` package.

## What You'll Learn

- Installing Ollama on macOS/Linux/Windows
- Pulling and managing models
- Basic chat with local models
- Listing available local models
- Benefits of running locally

## Popular Models

| Model | Size | Use Case |
|-------|------|----------|
| `llama3.2` | 2-3GB | General purpose (default) |
| `llama3.2:1b` | ~1GB | Fastest, low resource |
| `mistral` | ~4GB | Great quality/speed balance |
| `codellama` | ~4GB | Code generation |
| `phi3` | ~2GB | Microsoft's small model |
| `gemma2` | ~5GB | Google's open model |

## Why Ollama?

- **Privacy**: Data never leaves your machine
- **Offline**: Works without internet
- **Free**: No API costs, no rate limits
- **Fast**: No network latency

## Installation

First, you need to install Ollama on your local machine.

### macOS
```bash
# Using Homebrew
brew install ollama

# Or download from https://ollama.com
```

### Linux
```bash
curl -fsSL https://ollama.com/install.sh | sh
```

### Windows
Download the installer from [ollama.com](https://ollama.com)

## Pulling Models

Before using a model, you need to download it. Run these in your terminal:

```bash
# Pull the default model (Llama 3.2)
ollama pull llama3.2

# Pull a smaller/faster model
ollama pull llama3.2:1b

# Pull other models
ollama pull mistral
ollama pull codellama
ollama pull phi3
```

Models are downloaded once and cached locally (~2-8GB each).

## Starting Ollama

Ollama runs as a background service. It usually starts automatically, but you can start it manually:

```bash
ollama serve
```

The API runs at `http://localhost:11434` by default.

## Setup

**Note**: This notebook requires Ollama running locally. It won't work in Google Colab (which runs in the cloud).

Run this notebook on your local machine with Jupyter installed.

In [ ]:
# Install the package (if not already installed)
!pip install -q git+https://github.com/deepakdeo/python-llm-playbook.git

In [ ]:
# Check if Ollama is running
import requests

try:
    response = requests.get("http://localhost:11434/api/tags")
    if response.status_code == 200:
        models = response.json().get('models', [])
        print("Ollama is running!")
        print(f"Available models: {[m['name'] for m in models]}")
    else:
        print("Ollama responded but with an error")
except requests.exceptions.ConnectionError:
    print("Ollama is not running!")
    print("Start it with: ollama serve")

## 1. Basic Chat with Local Models

Using Ollama is just like using any other provider - no API key needed!

In [ ]:
from llm_playbook import OllamaClient

# Initialize the client (uses llama3.2 by default)
client = OllamaClient()

# Simple chat
response = client.chat("What is machine learning in one sentence?")
print(response)

In [ ]:
# Use a specific model
mistral_client = OllamaClient(model="mistral")

response = mistral_client.chat("Write a haiku about coding.")
print(response)

## 2. Listing Local Models

See what models you have downloaded.

In [ ]:
# List available models
models = client.list_models()

print("Installed models:")
for model in models:
    name = model.get('name', 'unknown')
    size = model.get('size', 0) / (1024**3)  # Convert to GB
    print(f"  - {name} ({size:.1f} GB)")

## 3. System Prompts

Control the model's behavior with system prompts.

In [ ]:
# Default response
response = client.chat("Explain APIs")
print("Default:")
print(response)
print()

In [ ]:
# With system prompt
response = client.chat(
    message="Explain APIs",
    system_prompt="You explain things like a pirate. Say 'Arrr' and use nautical terms."
)
print("As a pirate:")
print(response)

## 4. Multi-turn Conversations

Maintain context across multiple exchanges.

In [ ]:
from llm_playbook import ChatMessage

history = []
system = "You are a helpful Python tutor. Be concise."

# Turn 1
q1 = "What is a dictionary in Python?"
a1 = client.chat(q1, system_prompt=system, history=history)

print(f"Student: {q1}")
print(f"Tutor: {a1}\n")

history.append(ChatMessage(role="user", content=q1))
history.append(ChatMessage(role="assistant", content=a1))

In [ ]:
# Turn 2 - follows context
q2 = "How do I add a new key?"
a2 = client.chat(q2, system_prompt=system, history=history)

print(f"Student: {q2}")
print(f"Tutor: {a2}")

## 5. Streaming Responses

Stream tokens as they're generated for real-time output.

In [ ]:
print("Streaming: ", end="")

for token in client.stream("Write a limerick about Python."):
    print(token, end="", flush=True)

print()

## 6. Code Generation with CodeLlama

If you have CodeLlama installed, it's great for coding tasks.

In [ ]:
# Try CodeLlama if available
try:
    code_client = OllamaClient(model="codellama")
    
    response = code_client.chat(
        message="Write a Python function to check if a number is prime",
        system_prompt="You are a code assistant. Only output code, no explanations."
    )
    print(response)
except Exception as e:
    print(f"CodeLlama not available: {e}")
    print("Pull it with: ollama pull codellama")

## 7. Benefits of Running Locally

### Privacy
- Your data never leaves your machine
- Great for sensitive information
- No logging by third parties

### Cost
- Completely free to use
- No API costs or rate limits
- Use as much as you want

### Speed
- No network latency
- Fast for local hardware (especially with GPU)
- Works offline

### Hardware Requirements

| Model Size | RAM Required | GPU Optional |
|------------|--------------|-------------|
| 1-3B | 4-8GB | Helpful |
| 7-8B | 8-16GB | Recommended |
| 13B+ | 16-32GB | Highly recommended |
| 70B | 48GB+ | Required |

## 8. Comparing Local vs Cloud

| Feature | Local (Ollama) | Cloud (OpenAI, etc.) |
|---------|----------------|---------------------|
| Privacy | ✅ Full control | ❌ Data sent to servers |
| Cost | ✅ Free | ❌ Per-token pricing |
| Quality | ⚠️ Good for open models | ✅ Best models |
| Speed | ⚠️ Hardware dependent | ✅ Optimized infrastructure |
| Offline | ✅ Works offline | ❌ Requires internet |
| Setup | ⚠️ Requires installation | ✅ Just API key |

## Summary

You've learned:

1. **Installation**: Install Ollama on macOS, Linux, or Windows
2. **Models**: Pull models with `ollama pull <model>`
3. **Usage**: Same interface as cloud providers
4. **Benefits**: Privacy, cost, and offline capability

## Next Steps

- Try the [comparison notebook](06_comparison.ipynb) to see Ollama vs cloud providers
- Explore different models for different tasks
- Consider Ollama for privacy-sensitive applications